In [650]:
import pandas as pd 
import numpy as np 
import json
import cv2


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator # data augmentation
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping



import matplotlib.pyplot as plt 
%matplotlib inline 

# Loading the Training and the Test Sets

In [651]:
train = pd.read_json('train/processed/train.json')
test = pd.read_json('test/processed/test.json')

In [652]:
train.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [653]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1604 entries, 0 to 1603
Data columns (total 5 columns):
band_1        1604 non-null object
band_2        1604 non-null object
id            1604 non-null object
inc_angle     1604 non-null object
is_iceberg    1604 non-null int64
dtypes: int64(1), object(4)
memory usage: 75.2+ KB


In [654]:
train['inc_angle'].value_counts().head()

na         133
34.4721     23
42.5591     16
36.1061     15
33.6352     15
Name: inc_angle, dtype: int64

We right away notice that there are some NA valus in the inc_angle column 

In [655]:
train['inc_angle'].value_counts().head().index[0]

'na'

In [656]:
'''Replacing the NAs with 0s'''
train['inc_angle'] = train.inc_angle.replace(train['inc_angle'].value_counts().head().index[0], 0)

In [657]:
'''Validating the Replacements'''
train['inc_angle'].value_counts().head()

0.0000     133
34.4721     23
42.5591     16
33.6352     15
36.1061     15
Name: inc_angle, dtype: int64

In [658]:
train.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [659]:
train.is_iceberg.value_counts()

0    851
1    753
Name: is_iceberg, dtype: int64

In [660]:
'''Preprocessing the image to be fed into the model'''

b1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train["band_1"]])
b2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_2']])

a = (b1 - b1.mean()) / (b1.max() - b1.min())
b = (b2 - b2.mean()) / (b2.max() - b2.min())
# b3 = a + b
# c = (b3 - b3.mean()) / (b3.max() - b3.min())


# b1 = b1/255.
# b2 = b2/255.
X = np.stack([a,b], axis=3)

# X = np.dstack((a,b,c))
y = to_categorical(train['is_iceberg'])

In [661]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [662]:
X_train.shape

(1283, 75, 75, 2)

In [663]:
y_test.shape

(321, 2)

# Buidling a Simple Keras Model

In [670]:
'''Setting up the Model'''
model=Sequential()

'''Setting up the Layers'''
model.add(Conv2D(filters=15, kernel_size=(5,5), strides=(1,1), input_shape=(75,75,2), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=30, kernel_size=(4,4), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(filters=45, kernel_size=(4,4), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=60, kernel_size=(4,4), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(50, activation='relu'))

'''Setting up the Output'''
model.add(Dense(2, activation='sigmoid'))

In [671]:
'''Compiling the Model'''

optimizer = Adam(lr=0.001, decay=0.0)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=10, verbose=0, mode='auto')
callbacks_list = [earlystop]

model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics = ['accuracy'])


In [672]:
'''Fitting the model'''
model.fit(X_train, y_train, validation_data=[X_test, y_test], epochs=35,\
          verbose=1, batch_size=32, callbacks=callbacks_list)

Train on 1283 samples, validate on 321 samples
Epoch 1/35
1283/1283 [==============================] - 12s 9ms/step - loss: 0.6695 - acc: 0.5362 - val_loss: 0.6539 - val_acc: 0.5950
Epoch 2/35
1283/1283 [==============================] - 8s 6ms/step - loss: 0.6170 - acc: 0.6274 - val_loss: 0.5686 - val_acc: 0.6137
Epoch 3/35
1283/1283 [==============================] - 8s 6ms/step - loss: 0.5555 - acc: 0.6843 - val_loss: 0.5389 - val_acc: 0.7009
Epoch 4/35
1283/1283 [==============================] - 8s 6ms/step - loss: 0.4624 - acc: 0.7677 - val_loss: 0.4162 - val_acc: 0.8131
Epoch 5/35
1283/1283 [==============================] - 8s 6ms/step - loss: 0.3730 - acc: 0.8277 - val_loss: 0.3989 - val_acc: 0.8349
Epoch 6/35
1283/1283 [==============================] - 8s 6ms/step - loss: 0.3420 - acc: 0.8332 - val_loss: 0.3845 - val_acc: 0.8349
Epoch 7/35
1283/1283 [==============================] - 8s 6ms/step - loss: 0.3948 - acc: 0.8145 - val_loss: 0.4349 - val_acc: 0.7882
Epoch 8/35
128

In [673]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_193 (Conv2D)          (None, 71, 71, 15)        765       
_________________________________________________________________
max_pooling2d_186 (MaxPoolin (None, 35, 35, 15)        0         
_________________________________________________________________
dropout_41 (Dropout)         (None, 35, 35, 15)        0         
_________________________________________________________________
conv2d_194 (Conv2D)          (None, 32, 32, 30)        7230      
_________________________________________________________________
max_pooling2d_187 (MaxPoolin (None, 16, 16, 30)        0         
_________________________________________________________________
dropout_42 (Dropout)         (None, 16, 16, 30)        0         
_________________________________________________________________
conv2d_195 (Conv2D)          (None, 13, 13, 45)        21645     
__________

# Getting Ready for some predictions 

In [674]:
'''Preprocessing the test image to be fed into the model'''
b1_test = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test["band_1"]])
b2_test = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test['band_2']])

# b1_test = b1_test/255.
# b2_test = b2_test/255.

b1_test = (b1_test - b1_test.mean()) / (b1_test.max() - b1_test.min())
b2_test = (b2_test - b2_test.mean()) / (b2_test.max() - b2_test.min())

# X = np.stack([a,b], axis=3)



In [675]:
X_test_submission = np.stack([b1_test,b2_test], axis=3)

In [676]:
pred = model.predict_classes(X_test_submission)

In [677]:
pred[0:10]

array([0, 1, 0, 1, 0, 0, 0, 0, 0, 0])

In [678]:
submit_df = pd.DataFrame({'id': test['id'], 'is_iceberg': pred})
submit_df.to_csv('naive_submission.csv', index=False)